<a href="https://colab.research.google.com/github/peterm-m/03MARI---Algoritmos-de-Optimizacion---2019/blob/master/Pedro_Martinez_Moreno_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad Guiada 3 

### Pedro Martinez Moreno

#### [link Google Colaboratory](https://colab.research.google.com/drive/1T_mKBZagbVx8XNs-J-rguvnZphBod7yA) 

#### [link GitHub](https://github.com/peterm-m/03MARI---Algoritmos-de-Optimizacion---2019)
 


## Problema del ajente viajero 

### Preparaciones previas: 

In [0]:
!pip install requests         #descargamos conjunto de datos e intalamos librerias

import urllib.request

file='swiss42.tsp'

urllib.request.urlretrieve('http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp',file)

!pip install tsplib95



In [0]:
import tsplib95
import random
from math import e
#cargamos datos.
problem=tsplib95.load_problem(file)

nodos=list(problem.get_nodes())

aristas=list(problem.get_edges())



### Funciones necesarias para problema TSP.

In [0]:
#una solucion es una secuencia ordenada de nodos sin repetición
def crear_solucion(nodos):
  solucion=[0]
  for i in range(len(nodos)-1):
    solucion=solucion+[random.choice(list(set(nodos)-set({0})-set(solucion)))] # elegimos nodos de forma aleatoria sin repeticion
  return solucion


def distancia(a,b,problema):
  if a==b:
    return 10e-100 #evita divisiones por cero
  else:
    return problema.wfunc(a,b) # funcion distancia

def distancia_total(solucion,problema):# la distancia total es la suma de todas las distancias
  distancia_total=0
  for i in range(len(solucion)-1):
    distancia_total+=distancia(solucion[i],solucion[i+1],problema)
  return distancia_total+distancia(solucion[len(solucion)-1],solucion[0],problema)

#comprobamos que funcionan  todas las rutinas
ejemplo=crear_solucion(nodos)
print(ejemplo)
print(distancia(1,2,problem))
print(distancia_total(ejemplo,problem))


[0, 13, 23, 19, 33, 34, 35, 9, 2, 40, 21, 27, 32, 5, 17, 41, 20, 28, 37, 11, 38, 15, 4, 16, 29, 7, 39, 6, 26, 31, 1, 36, 3, 30, 24, 12, 14, 8, 10, 22, 25, 18]
34
4973


###  Busqueda aleatoria.

In [0]:
def busqueda_aleatoria(problema , n):

  nodos=list(problema.get_nodes())

  mejor_solucion=list()
  mejor_distancia=10e100

  for i in range(n):
    solucion =crear_solucion(nodos) # genera solucion aleatoria
    distancia_solucion=distancia_total(solucion,problema)
    if distancia_solucion<mejor_distancia: #si es mejor que la solucion actual actualizamos esta.
      mejor_solucion=solucion
      mejor_distancia=distancia_solucion
  print(solucion)
  print(mejor_distancia)
  return mejor_solucion

sol=busqueda_aleatoria(problem,1000)

[0, 23, 2, 25, 4, 7, 8, 39, 26, 11, 19, 41, 31, 36, 14, 16, 1, 24, 5, 28, 15, 17, 27, 13, 18, 30, 35, 20, 12, 29, 10, 33, 9, 37, 40, 3, 32, 6, 34, 21, 22, 38]
3970


### Busqueda local.

In [0]:
def generar_vecina(solucion):
#Una solucion vecina es una en la cual solo dos elementos estan intercambiados.
  mejor_solucion=list()
  mejor_distancia=10e100

  for i in range(1,len(solucion)-1):
    for j in range(i+1,len(solucion)):
      vecina=solucion[:i]+[solucion[j]]+solucion[i+1:j]+[solucion[i]]+solucion[j+1:] # permutacion
      distancia_vecina=distancia_total(vecina,problem)
      if distancia_vecina <= mejor_distancia: # si con la permutación la distancia es mejor actualizamos la solucion
        mejor_distancia = distancia_vecina
        mejor_solucion=vecina
  return mejor_solucion

def busqueda_local(problema,n):
  mejor_solucion=list()
  mejor_distancia=10e100

  solucion_referencia=crear_solucion(nodos) # genera una solucion
  for i in range(n):
    vecina = generar_vecina(solucion_referencia)
    distancia_vecina=distancia_total(vecina,problema) # busca entere los vecinos
    if distancia_vecina < mejor_distancia:  # si es mejor, actualizamos la solucion
      mejor_solucion=vecina
      mejor_distancia=distancia_vecina
    solucion_referencia=vecina

  print(mejor_solucion)
  print(mejor_distancia)

  return mejor_solucion

sol=busqueda_local(problem,100)





[0, 7, 19, 13, 10, 21, 24, 40, 23, 41, 8, 29, 30, 32, 1, 6, 4, 3, 27, 2, 28, 9, 39, 22, 38, 34, 33, 20, 35, 36, 31, 17, 37, 15, 16, 14, 12, 11, 25, 18, 26, 5]
1696


### Recocido Simulado.



In [0]:
import math

def generar_vecina_aleatoria(solucion): # generamos vecino intercambiando dos indices
  i=random.choice(range(1,len(solucion)))
  j=random.choice(list(set(range(1,len(solucion)))-{i}))
  if j<i:
    i,j=j,i
  vecina=list(solucion[:i]+[solucion[j]]+solucion[i+1:j]+[solucion[i]]+solucion[j+1:])
  return vecina


def probabilidad(T,d):
  if random.random()<=math.exp(-1*d/T) # distribución de Bolzman 
    return True                        # esta nos permitira escojer soluciones
  else:                                # cercanas a la solucion actual
    return False
  
def bajar_temperatura(T): # reduce la temperatura
  return T*0.999

def recocido_simulado(problema,T=1000):

  solucion_referencia  = crear_solucion(nodos) #solucion inicial
  distancia_referencia = distancia_total(solucion_referencia,problema) # distancia solucion inicial

  mejor_solucion=list()
  mejor_distancia=10e100

  while T > 1:

    vecina = generar_vecina_aleatoria(solucion_referencia)

    distancia_vecina=distancia_total(vecina,problema)

    if distancia_vecina < mejor_distancia:

      mejor_solucion=vecina
      mejor_distancia=distancia_vecina

    p=probabilidad(T,abs(distancia_referencia-distancia_vecina))
    if distancia_vecina < distancia_referencia or p:
      solucion_referencia=vecina
      distancia_referencia=distancia_vecina

    T=bajar_temperatura(T)
  print(mejor_distancia,'\n',mejor_solucion)
  return mejor_solucion,mejor_distancia

sol,dis=recocido_simulado(problem,100000)

1764 
 [0, 3, 4, 8, 25, 11, 12, 18, 26, 6, 34, 33, 35, 36, 16, 15, 37, 17, 31, 20, 32, 27, 2, 1, 7, 14, 19, 13, 5, 10, 41, 23, 39, 22, 38, 24, 40, 21, 9, 29, 30, 28]


## Optimizacion por enjambre: Colonia de Hormigas.

In [0]:
def otro_nodo(problema,H,T):
  nodos=list(problema.get_nodes())
  return random.choice(list(set(range(1,len(nodos)))-set(H))) #eleccion totalmente aleatoria, poco eficiente.

def incrementar_feromona(problema,T,H):
  for i in range(len(H)-1): 
    T[H[i]][H[i+1]]+=1000/distancia_total(H,problema) # aumenta feromona 
  return T

def evaporar_feromona(T):
  T=[[max(T[i][j]-0.3,1) for i in range(len(nodos))] for j in range(len(nodos))] # evapora feromona
  return T

def hormigas(problema,n):

  nodos=list(problema.get_nodes())
  aristas=list(problema.get_edges())
  T=[[1 for _ in range(len(nodos))] for _ in range(len(nodos))] #inicializamos feromona
                                                                #todos en 1 para evitar divisiones por cero en el futuro,
                                                                # esto se puede hacer, ya que lo queimporta es la diferencia de feromona entre caminos no
                                                                #la cantidad total.       

  hormiga=[[0] for _ in range(n)]#inicializamos recorrido hormigas
  
  for h in range(n):
    for i in range(len(nodos)-1):
      nuevo_nodo=otro_nodo(problema,hormiga[h],T) #elegimos nuevo nodo
      hormiga[h].append(nuevo_nodo)
      
      #actualizamos feromona
    T=incrementar_feromona(problema,T,hormiga[h]) 
    T=evaporar_feromona(T)

  #comprobamos mejor solucion
  mejor_solucion=list()
  mejor_distancia=10e100

  for h in range(n):
    distancia_actual=distancia_total(hormiga[h],problema)
    if distancia_actual<mejor_distancia:
      mejor_solucion=hormiga[h]
      mejor_distancia= distancia_actual
  print(mejor_solucion,mejor_distancia)
  return mejor_distancia,mejor_solucion

sol,dist=hormigas(problem,10000)

[0, 19, 37, 14, 13, 4, 21, 40, 22, 15, 16, 28, 3, 35, 7, 27, 8, 10, 25, 18, 5, 23, 39, 24, 38, 9, 6, 32, 41, 2, 36, 1, 11, 30, 34, 33, 31, 29, 20, 26, 12, 17] 3680


### Colonia de Hormigas: Mejora.

In [0]:
import numpy as np

def otro_nodo_mejorado(problema,H,T,a=0,b=0):

  nodos=list(problema.get_nodes())
  nodos_admisibles=list(set(range(1,len(nodos)))-set(H)) # ciudades no visitadas
  nodo_actual=H[-1] #ciudad actual

  #funcion de probabilidad para elegir nuevo nodo en funcion de la feromona y de la distancia

  denominador=sum(list(map( lambda nodo : pow(T[nodo_actual][nodo],a)*pow(distancia(nodo_actual,nodo,problema)**(-1),b), nodos_admisibles))) 
  probabilidad=list()
  for j in nodos_admisibles:
    probabilidad.append(pow(T[nodo_actual][j],a)*pow(distancia(nodo_actual,j,problema)**(-1),b)/denominador)
  return np.random.choice(np.array(nodos_admisibles),p=np.array(probabilidad))


def incrementar_feromona(problema,T,H,Q=1000):
  for i in range(len(H)-1):
    T[H[i]][H[i+1]]+=Q/distancia_total(H,problema) # de forma mas general pordemos tener un parametro de incremento de feromonas a
  return T

def evaporar_feromona(T,rho=0.7): #de forma mas general la feromona se evapora de forma proporcional a un factor de evaporación y la cantidad total de feromona depositada
  T=[[max(T[i][j]*(1-rho),1) for i in range(len(nodos))] for j in range(len(nodos))]
  return T




def hormigas_mejorado(problema,a,b,Q,rho,n):

  nodos=list(problema.get_nodes())

  aristas=list(problema.get_edges())

  T=[[min(1/distancia(i,j,problema),1) for i in range(len(nodos))] for j in range(len(nodos))] #inicializamos feromona para que 
                                                                                               #empiece con mayor feromona las ciudades mas cercanas

  hormiga=[[0] for _ in range(n)]

  for h in range(n):
    for i in range(len(nodos)-1):
      nuevo_nodo=otro_nodo_mejorado(problema,hormiga[h],T,a,b)
      hormiga[h].append(nuevo_nodo)

    T=incrementar_feromona(problema,T,hormiga[h],Q)

    T=evaporar_feromona(T,rho) 

  mejor_solucion=list()
  mejor_distancia=10e100

  for h in range(n):
    distancia_actual=distancia_total(hormiga[h],problema)
    if distancia_actual<mejor_distancia:
      mejor_solucion=hormiga[h]
      mejor_distancia= distancia_actual
  print(mejor_solucion,mejor_distancia)
  return mejor_distancia,mejor_solucion

sol,dist=hormigas_mejorado(problem,0.5,2,1000,0.3,10000)

[0, 3, 27, 2, 26, 5, 29, 4, 7, 1, 28, 30, 18, 13, 19, 14, 16, 15, 37, 17, 31, 35, 36, 10, 25, 11, 12, 41, 23, 9, 8, 39, 21, 40, 24, 22, 38, 34, 33, 20, 32, 6] 1846
